## Use OpenCL To Add Two Random Arrays (This Way Hides Details)


In [1]:
import pyopencl as cl
import pyopencl.array as pycl_array
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


pyopencl cho phép viết kernel OpenCL và chạy trên GPU/CPU.

pyopencl.array giúp quản lý dữ liệu như numpy array nhưng nằm trên GPU.

In [2]:
context = cl.create_some_context()
queue = cl.CommandQueue(context)


Context: môi trường liên kết với platform + device (GPU/CPU).

Queue: nơi ta gửi lệnh cho device (copy data, chạy kernel…).

In [3]:
a = pycl_array.to_device(queue, np.random.rand(50000).astype(np.float32))
b = pycl_array.to_device(queue, np.random.rand(50000).astype(np.float32))
c = pycl_array.empty_like(a)


to_device; copy dữ liệu từ numpy sang gpu
empty_like: tạo vùng nhớ trống trên gpu có kích thước bằng mảng a
a, b, c đều trên GPU memory

In [4]:
program = cl.Program(context, """
__kernel void sum(__global const float *a, __global const float *b, __global float *c)
{ int i = get_global_id(0); 
   c[i] = a[i] + b[i];
}""").build()

program.sum(queue, a.shape, None, a.data, b.data, c.data)

print ("a : {}" .format(a))
print ("b : {}" .format(b))
print ("c : {}" .format(c))

                     

a : [0.30065638 0.04990655 0.6094854  ... 0.775733   0.24715574 0.90530264]
b : [0.41555896 0.76297724 0.7838401  ... 0.96351594 0.13746446 0.4645607 ]
c : [0.7162154 0.8128838 1.3933256 ... 1.739249  0.3846202 1.3698633]


__kernel : định nghĩa hàm chạy trên GPU.
__global: dữ liệu trong bộ nhớ GPU
get_global_id(0) trả về chỉ số thread toàn cục (ở đây mỗi thread xử lý 1 phần tử).
Mỗi thread tính c [i] = a[i] + b[i]
cl.Program biến dịch kernel thành code cho GPU
program.sum(...) gọi kernel:
queue = hàng đợi thực thi.
a.shape = global size (ở đây 50000 thread).
None = local size (để driver tự chọn).
a.data, b.data, c.data = buffer thực sự trên GPU.
